In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [3]:
spark = (
    SparkSession.builder
                .appName("Stack Overflow Data Wrangling2")
                .config("spark.jars", "/home/pry/codes/blossom/jars/postgresql-42.2.8.jar") 
                .getOrCreate()
)

In [4]:
# pg_url = 'jdbc:postgresql://PG_SERVER_IP/db_name?user=PG_USER&password=PASSWORD'
# pg_url = 'jdbc:postgresql://localhost:5432/stackoverflow?user=postgres&password=postgres',
pg_url = 'jdbc:postgresql://localhost:5432/postgres'

pg_properties = {
    "driver": "org.postgresql.Driver",
    "schema": "stackoverflow",
    "user": "postgres",
    "password": "postgres"
}

In [5]:
df_qtns = spark.read.jdbc(url=pg_url,table='stackoverflow.questions',properties=pg_properties)
df_qtns = df_qtns.withColumnRenamed('id', 'qtn_id') 
# df_qtns.head()

In [6]:
df_ans = spark.read.jdbc(url=pg_url,table='stackoverflow.answers',properties=pg_properties)
# df_qtns.head()

In [7]:
df_users = spark.read.jdbc(url=pg_url,table='stackoverflow.users',properties=pg_properties)

In [40]:
df_users = df_users.withColumnRenamed('id', 'userz_id') 
df_users = df_users.withColumnRenamed('created_at', 'user_created_at') 

# dfu = df_users
df_users.columns

['userz_id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'user_created_at',
 'updated_at']

In [20]:
# df_users.select('location').distinct().show(3)

### 1 Select users from only one country of your choosing.

In [21]:
user_loc = df_users.filter(col('location').contains('Beijing, China'))
# user_loc.take(15)

### 2 Extract the country and city into new columns.

In [22]:
user_loc = user_loc\
                .withColumn('city', F.split(user_loc['location'], ',')[0])\
                .withColumn('country', F.split(user_loc['location'], ',')[1])

In [23]:
# user_loc.take(5)

### 3 Join this with the questions

    only pick questions with at least 20 view_counts.

In [60]:
# df_qtns.columns

In [57]:
# user_loc.columns

In [54]:
# join_user_qtns = user_loc.join(user_loc, df_qtns.user_id==user_loc.userz_id, how='inner')

joined = user_loc.join(df_qtns, df_qtns["user_id"] == user_loc["userz_id"])
joined.head()

Row(userz_id=1120704, display_name='Shaun Xu', reputation='3571', website_url='http://blog.shaunxu.me', location='Dongcheng, Beijing, China', about_me="<p>Shaun, a semi-geek, clumsy developer, passionate speaker and incapable architect with about 10 years experience in .NET. He hopes to prove that software development is art rather than manufacturing. He's into cloud computing platform and technologies (Windows Azure, Aliyun) as well as WCF and ASP.NET MVC. Recently he's falling in love with JavaScript and Node.js.\nCurrently Shaun is working at IGT Technology Development (Beijing) Co., Ltd. as the architect responsible for product framework design and development.</p>\n", views=439, up_votes=27, down_votes=0, image_url='', user_created_at=datetime.datetime(2011, 12, 29, 8, 1, 22), updated_at=datetime.datetime(2019, 10, 12, 10, 33, 41), city='Dongcheng', country=' Beijing', qtn_id=55804034, user_id=1120704, title='`keyof` reported compile error when using generic type', body="<p>First 

In [55]:
# 

### 4 Join the answers to the results of (3).

In [58]:
# df_ans.columns

['id',
 'user_id',
 'question_id',
 'body',
 'score',
 'comment_count',
 'created_at']

In [64]:
# df_qtns.columns

In [62]:
df_ans = df_ans.withColumnRenamed('id', 'ans_id') 
df_ans = df_ans.withColumnRenamed('created_at', 'ans_created_at') 
df_ans = df_ans.withColumnRenamed('body', 'ans_body') 
df_ans = df_ans.withColumnRenamed('score', 'ans_score') 
df_ans = df_ans.withColumnRenamed('user_id', 'ans_user_id') 
df_ans = df_ans.withColumnRenamed('comment_count', 'ans_comment_count') 

In [63]:
df_ans.columns

['ans_id',
 'ans_user_id',
 'question_id',
 'ans_body',
 'ans_score',
 'ans_comment_count',
 'ans_created_at']

In [69]:
df_ans2 = df_ans.filter(df_ans.ans_user_id.isNotNull())

In [70]:
joined_ans = joined.join(df_ans2, joined["userz_id"] == df_ans2["ans_user_id"])

In [73]:
# joined_ans.columns

# 3. DATA LOADING:

In [76]:
joined_ans.write.format("jdbc").options(
    url='jdbc:postgresql://localhost/postgres',
    driver='org.postgresql.Driver',
    user='postgres',
    password='postgres',
    dbtable='stackoverflow_filtered.results'
).save(mode='append')


Py4JJavaError: An error occurred while calling o773.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 20, localhost, executor driver): org.postgresql.util.PSQLException: Ran out of memory retrieving query results.

## DIFFERENCE VIEW, MATERIALIZED

#### VIEW
a view does not store data physically 

### MATERIALIZED VIEW
allow you to store result of a query physically